### Import libraries

Import libraries, set parameters, and import classes

In [ ]:
import numpy as np
import tensorflow as tf
import pickle
np.random.seed(1)
tf.random.set_seed(1)
tf.keras.backend.set_floatx('float32')

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (3,3)
plt.rcParams.update({'font.size': 13})

In [ ]:
from __lib__.NN_class import NeuralNetwork
from __lib__.DataProcessing import PreProcessing

### Import datasets and pre-processing

In [ ]:
file = './data/3D_PlaneStrain_Elasticity_CSMA'
training_percentage = 0.6

In [ ]:
with open(file, 'rb') as f_obj:
    data = pickle.load(f_obj)

strain,stress,energy = data

del data

##### Split datasets  (tv=training+validation - test)

In [ ]:
PreProcessing = PreProcessing(training_percentage)

train_p,val_p,test_p = PreProcessing.split_dataset(strain,percentage=True)

print('# samples:', train_p+val_p+test_p,
      '\n# training:', train_p,
      '\n# validation:', val_p,
      '\n# test:', test_p)

# samples: 21000 
# training: 12600 
# validation: 4200 
# test: 4200


In [ ]:
strain_tv,strain_test = PreProcessing.split_dataset(strain)
stress_tv,stress_test = PreProcessing.split_dataset(stress)
energy_tv,energy_test = PreProcessing.split_dataset(energy)

##### Get normalization parameters

prm_E = [alpha_E, beta_E] # strain

prm_S = [alpha_S, beta_S] # stress

prm_F = [alpha_F, beta_F] # free energy

params = [prm_E,prm_S,prm_F]


if NN: Nnet.SetParams(params)

Else: TANNnet.SetParams(params)

##### Normalize datasets

In [ ]:
# Training and validation datasets

__strain_tv,__strain_test = PreProcessing.Normalize(
    [strain_tv,strain_test],prm_E,train_val=True)
__stress_tv,__stress_test = PreProcessing.Normalize(
    [stress_tv,stress_test],prm_S,train_val=True)
__energy_tv,__energy_test = PreProcessing.Normalize(
    [energy_tv,energy_test],prm_F,train_val=True)



NameError: name 'prm_E' is not defined

### Neural Networks

#### Build network

In [ ]:
umat_dim = 4 # dimensional space

hidden_NN_Stress = 6 # number of hidden layers in NN Evolution
material = 'elastic'

Nnet = NeuralNetwork(umat_dim,
                     material,
                     hidden_NN_Stress,
                     activation_NN_Stress='tanh',
                     ) # Initialize TANN class


params = [prm_E,prm_S] # regroup normalization parameters
Nnet.SetParams(params) # set normalization parameters in the model

# build TF model class
input_shape = [None,umat_dim]
Nnet.build(input_shape)

# print model's summary
print(Nnet.summary())

#### Define inputs and outputs

In [ ]:
output_tv = __stress_tv

output_test = __stress_test

input_tv = __strain_tv

input_test = __strain_test

#### Compile and train the network

In [ ]:
Nnet.compileNet()

hist_Nnet = Nnet.fit(input_tv,output_tv,
                           epochs = 1500,
                           validation_split=0.2,
                           batch_size = 200,
                           verbose = 2,
                           use_multiprocessing=True)

#### Evaluate losses

In [ ]:
print('Evaluate loss for training and validation datasets:'); Nnet.evaluate(input_tv,output_tv,verbose=2)
print('\nEvaluate loss for test datasets:'); Nnet.evaluate(input_test,output_test,verbose=2)


stress_loss = np.asarray(hist_Nnet.history['loss'])
stress_val_loss = np.asarray(hist_Nnet.history['val_loss'])
epochs = range(1, len(stress_loss) + 1)

#### Plot training curves

In [ ]:
fig, ax = plt.subplots(1)
ax.loglog(epochs, stress_loss, color='black', label='$\sigma$', linewidth=2, marker='o',markersize=0)
ax.loglog(epochs, stress_val_loss, color='black',  alpha=0.5,  linewidth=2, marker='o',markersize=0)


ax.set_xlabel('epochs')
ax.set_ylabel('Mean Absolute Error')
ax.legend(loc='best')
ax.set_ylim(5e-9,10)
ax.grid()

#### Predictions for the entire batch

In [ ]:
pred__stress = Nnet.predict(input_test)

# Plot
plt.plot(__stress_test,
         pred__stress,
         alpha=0.3,linewidth=0, marker='o',markersize=5)
plt.xlabel("$\sigma$")
plt.ylabel("$\sigma=NN(\\varepsilon)$")
plt.show()

#### Prediction point by point

In [ ]:
pred__stress_point = Nnet.predict_on_batch(np.array([input_test[0]]))
print('Predicted stress point', pred__stress_point)
print('Test stress point', __stress_test[0])

### Save model

In [ ]:
# filename = 'results/NN_Elasticity'
# Nnet.save_export(filename)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b33f54f1-f98d-4fed-99c0-2484f115edd9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>